In [2]:
import pandas as pd 
import numpy as np

In [3]:
# data import
balance_data = pd.read_csv('balance_data.csv')
nsfr_haircut = pd.read_csv('nsfr_haircut.csv')

In [4]:
# changing the type to float
pd.options.display.float_format = '{:,.2f}'.format

In [5]:
balance_data.head()

,NSFR_GROUP,BALANCE_GROUP,BALANCE_CATEGORY,BALANCE_SUB_CATEGORY,GAP,TOTAL_BALANCE_EUR
0,RSF,Bonds,Bonds,Level 1,<6m,"423,001,312.00"
1,RSF,Bonds,Bonds,Level 1,6 - 12m,"400,123,123.00"
2,RSF,Bonds,Bonds,Level 1,>12m,"160,312,999.00"
3,RSF,Bonds,Bonds,Level 2A,<6m,"19,229,999.00"
4,RSF,Bonds,Bonds,Level 2A,6 - 12m,"12,312,331.00"


In [6]:
nsfr_haircut.head()

,NSFR_GROUP,BALANCE_GROUP,BALANCE_CATEGORY,BALANCE_SUB_CATEGORY,GAP,NSFR_HAIRCUT
0,RSF,Bonds,Bonds,Level 1,<6m,0.00
1,RSF,Bonds,Bonds,Level 1,6 - 12m,0.00
2,RSF,Bonds,Bonds,Level 1,>12m,0.00
3,RSF,Bonds,Bonds,Level 2A,<6m,0.15
4,RSF,Bonds,Bonds,Level 2A,6 - 12m,0.15


In [7]:
def task_4(balance_data, nsfr_haircut):
    balance_data = balance_data.copy()
    nsfr_haircut = nsfr_haircut.copy()

In [8]:
# merging the data
merged_data = pd.merge(balance_data, nsfr_haircut, how='inner', on=['NSFR_GROUP', 'BALANCE_GROUP', 'BALANCE_CATEGORY', 'BALANCE_SUB_CATEGORY', 'GAP'])

In [9]:
merged_data.head()

,NSFR_GROUP,BALANCE_GROUP,BALANCE_CATEGORY,BALANCE_SUB_CATEGORY,GAP,TOTAL_BALANCE_EUR,NSFR_HAIRCUT
0,RSF,Bonds,Bonds,Level 1,<6m,"423,001,312.00",0.00
1,RSF,Bonds,Bonds,Level 1,6 - 12m,"400,123,123.00",0.00
2,RSF,Bonds,Bonds,Level 1,>12m,"160,312,999.00",0.00
3,RSF,Bonds,Bonds,Level 2A,<6m,"19,229,999.00",0.15
4,RSF,Bonds,Bonds,Level 2A,6 - 12m,"12,312,331.00",0.15


In [10]:
# calculating the NSFR ratio and dividing into groups

merged_data['product'] = merged_data['TOTAL_BALANCE_EUR'] * merged_data['NSFR_HAIRCUT']
RSF_sum = merged_data.query("NSFR_GROUP == 'RSF' | NSFR_GROUP == 'RSF_OBS'")['product'].sum()
ASF_sum = merged_data.query("NSFR_GROUP == 'ASF'")['product'].sum()
NSFR_dec = ASF_sum/RSF_sum

In [11]:
# formating for NSFR ratio
print('\n4. Task solution')
print('\nCurrent NSFR is {:.2%}'.format(NSFR_dec))
print('NSFR is greater than 100%, the bank has enough available stable funding to meet the required stable funding\n')


4. Task solution

Current NSFR is 185.21%
NSFR is greater than 100%, the bank has enough available stable funding to meet the required stable funding



In [12]:
# pivot table for daily balance by balance groups and time gaps
column_order = ['<6m', '6 - 12m', '>12m'] #to order the pivot columns
balance_by_gap = merged_data.pivot_table(index=['NSFR_GROUP', 'BALANCE_GROUP'],
                                         columns='GAP',
                                         values='TOTAL_BALANCE_EUR',
                                         aggfunc='sum',
                                         fill_value=0)
balance_by_gap1 = balance_by_gap.reindex(column_order, axis=1)

In [13]:
# weighted average NSFR haircuts by balance groups
weighted_haircuts = merged_data.pivot_table(index=['NSFR_GROUP', 'BALANCE_GROUP'],
                                            values='NSFR_HAIRCUT',
                                            # using numpy to aggregate with weighted average
                                            aggfunc=lambda rows: np.average(rows, weights=merged_data.loc[rows.index, 'TOTAL_BALANCE_EUR']))

In [74]:
print('\nDaily balance by balance groups and time gaps:')
display(balance_by_gap1)


Daily balance by balance groups and time gaps:


GAP                                               <6m        6 - 12m  \
NSFR_GROUP BALANCE_GROUP                                               
ASF        Deposits                  2,273,503,949.00 126,697,932.72   
           Other liabilities           137,694,805.70           0.00   
           Tier 1 and Tier 2 capital             0.00           0.00   
RSF        All other assets             70,168,404.38           0.00   
           Bonds                       694,067,391.07 420,100,046.94   
           Cash and CB Reserves        162,880,263.70           0.00   
           Encumbered assets            83,456,143.59  10,732,123.00   
           Loans                        92,794,608.93  96,560,585.96   
           Nostro and money market      70,168,404.38           0.00   
           Other assets                          0.00           0.00   
RSF_OBS    Credit facilities               300,999.99           0.00   
           Liquidity facilities         12,011,657.40           0.00   

GAP                                              >12m  
NSFR_GROUP BALANCE_GROUP                               
ASF        Deposits                     52,073,486.22  
           Other liabilities            65,490,777.04  
           Tier 1 and Tier 2 capital   372,215,414.08  
RSF        All other assets                      0.00  
           Bonds                       211,222,675.58  
           Cash and CB Reserves                  0.00  
           Encumbered assets            35,558,559.00  
           Loans                     1,065,028,597.48  
           Nostro and money market               0.00  
           Other assets                 14,938,560.60  
RSF_OBS    Credit facilities                     0.00  
           Liquidity facilities                  0.00

In [15]:
print('\nWeighted NSFR haircuts by balance groups:')
display(weighted_haircuts)



Weighted NSFR haircuts by balance groups:


NSFR_HAIRCUT
NSFR_GROUP BALANCE_GROUP                          
ASF        Deposits                           0.84
           Other liabilities                  0.32
           Tier 1 and Tier 2 capital          1.00
RSF        All other assets                   0.00
           Bonds                              0.12
           Cash and CB Reserves               0.00
           Encumbered assets                  1.00
           Loans                              0.80
           Nostro and money market            0.50
           Other assets                       0.85
RSF_OBS    Credit facilities                  0.05
           Liquidity facilities               0.05